# Introduction to A* Search Algorithm

The A* search algorithm is a central method for pathfinding and graph traversal, efficiently finding the most direct route from a starting point to a destination within a graph. Its applications range from guiding GPS routes to improving video game environments, due to its effective management of complex pathfinding tasks.

A* cleverly combines the strengths of Dijkstra's algorithm with those of the Greedy Best-First-Search. It adopts Dijkstra’s reliability, ensuring the identification of the shortest path under all conditions, as long as the heuristic it employs is admissible and thus, does not overestimate the distance to the goal. Simultaneously, A* benefits from the Greedy Best-First-Search's heuristic approach, focusing its search towards the goal and avoiding less promising paths unless absolutely necessary.


## How A* Operates

A* utilizes a priority queue to sort nodes for exploration, arranging them based on a cost function, f(n) = g(n) + h(n):

g(n) indicates the cost from the starting node to the current node n.

h(n) is the estimated cost from n to the goal, incorporating domain-specific knowledge to guide the search.

The algorithm continuously selects the node with the smallest f(n) value, maintaining this strategy until it reaches the goal. The effectiveness of the heuristic greatly impacts A*'s performance, with the ideal heuristic closely matching the actual cost without exceeding it.

### Key Attributes


Optimality: With an admissible heuristic, A* ensures the identification of the shortest path.

Efficiency: A* reduces the search area by wisely choosing nodes for exploration, speeding up the pathfinding process.

Adaptability: A* can accommodate a variety of pathfinding scenarios, handling different types of graphs and routing challenges.

## Project

In our project, we utilized the A* search algorithm to optimize city delivery routes, taking into account a city's dynamic traffic patterns. We visualized the city as a weighted graph where intersections serve as nodes and roads as connecting links. A distinctive feature of our model is the implementation of time-variable weights on these links to reflect real-time traffic changes.

This strategy models the variable travel times seen in city settings, where congestion levels—and consequently, link weights—vary, particularly during peak traffic times. The goal was to design a delivery route that not only minimizes travel time from the starting point to the destination but also actively responds to changing traffic conditions.

For this purpose, we designed a hypothetical city grid, a 5x5 layout, to represent a basic urban structure. Each grid cell, an intersection, connects directly to its neighbors, forming a network of roads whose travel times increase during heavy traffic periods, thus adding an extra layer of complexity to route optimization.

## Model

#### Sets
- $V$: Set of vertices (intersections), indexed by $i$.
- $E$: Set of edges (roads between intersections), indexed by $(i,j)$ where $i, j \in V$.

#### Parameters
- $w_{i,j,t}$: Weight of edge $(i,j)$ at time $t$, representing the travel time which varies with traffic congestion.
- $H$: Set of peak hours, during which travel times are increased.
- $P$: Penalty multiplier for travel times during peak hours.

#### Decision Variables
- $x_{i,j,t}$: Binary variable that equals 1 if the edge $(i,j)$ is included in the optimal path at time $t$, and 0 otherwise.

#### Objective Function
$$\min \sum_{(i,j) \in E} \sum_{t} w_{i,j,t} \cdot x_{i,j,t}$$
- The objective is to minimize the total travel time considering the time-dependent weights of the edges.

#### Constraints
1. **Flow Conservation:** Ensure that for every node (except the start and end nodes), the sum of incoming traffic equals the sum of outgoing traffic.
$$\sum_{j:(i,j) \in E} x_{i,j,t} - \sum_{j:(j,i) \in E} x_{j,i,t} = 0, \quad \forall i \in V \setminus \{start, end\}, \forall t$$

2. **Start and End Nodes:** There should be exactly one outgoing edge from the start node and one incoming edge to the end node.
$$\sum_{j:(start,j) \in E} x_{start,j,t} = 1, \quad \sum_{j:(j,end) \in E} x_{j,end,t} = 1$$

3. **Time Dependency:** Adjust the weight of the edges during peak hours.
$$w_{i,j,t} = w_{i,j,t} \cdot P, \quad \forall (i,j) \in E, \forall t \in H$$

4. **Binary Variable Constraint:** Ensures that $x_{i,j,t}$ can only take the values 0 or 1.
$$x_{i,j,t} \in \{0,1\}, \quad \forall (i,j) \in E, \forall t$$

In [1]:
import numpy as np
import heapq

# Constants
PEAK_HOURS = [(8, 9), (17, 18)]  # (start, end) hours for morning and evening peaks
BASE_TIME = 1  # Base travel time for all edges
PEAK_TIME_MULTIPLIER = 1.2  # Multiplier for travel times during peak hours

# Function to determine if the current time is in peak hours
def is_peak_hour(hour):
    return any(start <= hour < end for start, end in PEAK_HOURS)

# Manhattan distance heuristic for A*
def manhattan_distance(start, goal):
    return abs(start[0] - goal[0]) + abs(start[1] - goal[1])

# Time-dependent travel time based on peak hours
def travel_time(current_hour):
    return BASE_TIME * PEAK_TIME_MULTIPLIER if is_peak_hour(current_hour % 24) else BASE_TIME

# Generate a 5x5 grid graph
def generate_grid_graph(n=5):
    graph = {}
    for x in range(n):
        for y in range(n):
            neighbors = []
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < n and 0 <= ny < n:
                    neighbors.append(((nx, ny), BASE_TIME))  # Base time will be adjusted dynamically
            graph[(x, y)] = neighbors
    return graph

# A* search algorithm
def a_star(graph, start, goal, start_time):
    queue = [(0 + manhattan_distance(start, goal), 0, start_time, start, [start])]
    visited = set()
    
    while queue:
        # Pop the node with the lowest f(x) = g(x) + h(x)
        _, cost, current_time, current, path = heapq.heappop(queue)
        if current in visited:
            continue
        visited.add(current)
        if current == goal:
            return path, cost
        
        for neighbor, _ in graph[current]:
            if neighbor in visited:
                continue
            next_cost = cost + travel_time(current_time)
            heapq.heappush(queue, (next_cost + manhattan_distance(neighbor, goal), next_cost, current_time + 1, neighbor, path + [neighbor]))
    
    return None, float('inf')  # Path not found

# Generate the city grid graph
grid_graph = generate_grid_graph()

# Example usage
start_node = (0, 0)
end_node = (4, 4)
start_hour = 8  # Example starting at 8 AM

path, total_cost = a_star(grid_graph, start_node, end_node, start_hour)
print(f"Optimal Path: {path}, Total Travel Time: {total_cost}")


Optimal Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)], Total Travel Time: 8.2
